# uv 가상환경 -> ipynb 설정
1. uv venv
2. .venv\Scripts\activate
3. uv sync
4. uv sync --dev
5. 커널에서 가상환경 선택

이게 될 때도 있고 안될 때가 있음;; 잘 모르겠네

In [1]:
!uv pip list

Package                      Version     Editable project location
---------------------------- ----------- ---------------------------------------
aiohappyeyeballs             2.6.1
aiohttp                      3.12.13
aiohttp-retry                2.9.1
aiosignal                    1.3.2
annotated-types              0.7.0
anthropic                    0.54.0
anyio                        4.9.0
asttokens                    3.0.0
asyncstdlib-fw               3.13.2
attrs                        25.3.0
beautifulsoup4               4.13.4
betterproto-fw               2.0.3
black                        25.1.0
blockbuster                  1.5.24
cachetools                   5.5.2
certifi                      2025.6.15
cffi                         1.17.1
charset-normalizer           3.4.2
click                        8.2.1
cloudpickle                  3.1.1
colorama                     0.4.6
comm                         0.2.2
cryptography                 44.0.3
dataclasses-json             0.6.

Using Python 3.13.3 environment at: C:\cursor\DealMakers\langgraph_baseline\.venv


In [2]:
# 여기서는 안됨... 이제 되네?
from google import genai


In [3]:
pwd

'c:\\cursor\\DealMakers\\langgraph_baseline\\notebook'

### 라이브러리 Import

In [4]:
import asyncio # 비동기 처리를 위한 모듈
import os # 환경변수 관리를 위한 모듈
import sys # 시스템 관련 모듈
import json # 데이터 처리를 위한 모듈
from contextlib import AsyncExitStack # 비동기 컨텍스트 관리를 위한 모듈
from typing import Optional, List  # 타입 힌트를 위한 모듈

# ------------------------------------------------------------
# MCP Client Imports
# ------------------------------------------------------------
from mcp import ClientSession, StdioServerParameters # MCP 클라이언트 세션 모듈
from mcp.client.stdio import stdio_client # MCP 클라이언트 표준 입출력 모듈


# ------------------------------------------------------------
# Agent and LLM Imports
# ------------------------------------------------------------
from langchain_mcp_adapters.tools import load_mcp_tools # MCP 툴 로드 모듈
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI


# ------------------------------------------------------------
# Environment Setup
# ------------------------------------------------------------
from dotenv import load_dotenv
load_dotenv()





True

### LLM

In [5]:
# LLM Instantiation
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash	", 
    temperature=0,
    max_retries=2, # 최대 2번 재시도
    api_key=os.getenv("GOOGLE_API_KEY"),
)


# MCP Server

#### MCP Server Script Argument

Command Line(명령줄)에서 프로그램을 실행할 때 추가적인 정보를 받아 처리하는 부분


In [4]:
if len(sys.argv) < 2:
    print("Usage: python client_langchain_google_genai_bind_tools.py <path_to_server_script>")
    sys.exit(1)
        # 프로그램 즉시 종료
server_script = sys.argv[1]

~.py라는 파이썬 스크립트를 실행할 때, 반드시 서버 스크립트의 경로를 인자로 전달해야하는 것을 강제하는 역할

## MCP Server Parameters
- Configure parameters to launch the MCP server.

In [ ]:
server_params = StdioServerParameters(
    command="python" if server_script.endswith(".py") else "node",
        # server_script 변수에 저장된 파일 이름이 .py로 끝나는지 확인해서 맞다면 -> "python" 아니면 -> "node"
    args=[server_script],
        # server_script ( = 실행할 스크립트 파일 경로)를 리스트에 담아 전달
)

# 활성 MCP 세션을 보유할 전역 변수
# 도구 어댑터에서 사용하기 위한 "세션" 속성을 가진 간단한 홀더
mcp_client = None


# MCP Client
- 미리 server를 실행시켜놔야함.

### 공식 문서

In [8]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent


server_params = StdioServerParameters(
    command="python",
    # args=["/path/to/server_script.py"],
    args=["C:\cursor\DealMakers\langgraph_baseline\notebook\server.py"],
)



async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)
        
        # Create and run the agent
        agent = create_react_agent(llm, tools)
        agent_response = await agent.ainvoke({"messages": "Hello, how are you?"})



NotImplementedError: 

In [ ]:
for message in agent_response['messages']:
    message.pretty_print()

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic
# from utils import aream_graph


# RAG 서버를 위한 StdIO 서버 파라미터 설정
server_params = StdioServerParameters(
    command="python",
    # args=["/path/to/server_script.py"],
    args=["C:\\cursor\\DealMakers\\langgraph_baseline\\notebook\\server\\mcp_rag.py"],
)

# StdIO 클라이언트를 사용하여 RAG 서버와 통신
async with stdio_client(server_params) as (read, write):
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()

        # MCP 도구 로드 (여기서는 retriever 도구)
        tools = await load_mcp_tools(session)

        # 에이전트 생성 및 실행
        agent = create_react_agent(llm, tools)

        # 에이전트 응답 스트리밍
        agent_response = await agent.ainvoke({"messages": "Hello, how are you?"})


NotImplementedError: 

## Agent

### 응용 예시

In [5]:
# Custom JSON Encoder

class CustomEncoder(json.JSONEncoder):
    """
    Custom JSON encoder that handles objects with a 'content' attribute.
    
    If an object has a 'content' attribute, it returns a dictionary with the object's type and its content.
    Otherwise, it falls back to the default encoding.
    """
    def default(self, o):
        if hasattr(o, "content"):
            return {"type": o.__class__.__name__, "content": o.content}
        return super().default(o)

In [8]:
async def run_agent():
    """
    Connect to the MCP server, load MCP tools, create a React agent, and run an interactive chat loop.
    
    Steps:
      1. Open a stdio connection to the MCP server.
      2. Create and initialize an MCP session.
      3. Store the session in a global holder (mcp_client) for tool access.
      4. Load MCP tools using load_mcp_tools.
      5. Create a React agent using create_react_agent with the LLM and loaded tools.
      6. Enter an interactive loop: for each user query, invoke the agent asynchronously using ainvoke,
         then print the response as formatted JSON using our custom encoder.
    """
    global mcp_client
    async with stdio_client(server_params) as (read, write):
            # 1. server_params에 지정된 명령어(python server_script.py)를 사용해 MCP 서버를 별도의 프로세스로 실행
            # 2. 실행된 서버 프로세스와 현재 클라이언트 프로세스 사이에 표준 입출력(stdio) 파이프를 연결합니다. 즉, 클라이언트가 터미널에 텍스트를 쓰는 것처럼 서버에 데이터를 보낼 수 있고, 서버가 터미널에 출력하는 텍스트를 클라이언트가 읽을 수 있게 됨
            # 3. 이 통신 파이프를 제어할 수 있는 두 개의 객체, read(서버로부터 데이터를 읽는 통로)와 write(서버에게 데이터를 쓰는 통로)를 반환
            # 클라이언트 프로그램이 MCP 서버와 통신하려면 가장 먼저 물리적인 연결 통로가 필요, 이 코드는 그 통로를 만들고 관리하는 역할을 합니다. async with 구문은 이 블록의 실행이 끝나면 서버 프로세스를 자동으로 종료하고 연결을 깔끔하게 닫아주는 역할을 보장하여, 프로그램이 비정상적으로 종료되는 것을 방지
        async with ClientSession(read, write) as session:
                # read와 write는 단순히 바이트(byte) 데이터를 주고받는 역할만 하지만, ClientSession 객체는 "초기화 메시지 보내기", "도구 목록 요청하기" 등 의미 있는 MCP 통신 명령을 수행할 수 있는 고수준의 인터페이스를 제공
                # 서버와 단순히 데이터를 주고받는 것만으로는 부족합니다. 정해진 약속(프로토콜)에 따라 메시지를 구조화하고, 요청과 응답을 관리해야 합니다. ClientSession은 이 복잡한 프로토콜 처리를 대신해주는 전문가 객체입니다. 개발자는 session.initialize()처럼 간단한 명령만 내리면, 내부적으로 복잡한 메시지 포맷팅과 전송을 이 객체가 알아서 처리
            await session.initialize() # 세션 초기화
                # session 객체를 통해 MCP 서버에게 "통신을 시작하자"는 공식적인 초기화(initialization) 요청
                # 클라이언트와 서버가 본격적으로 데이터를 교환하기 전에, 서로 통신할 준비가 되었는지 확인하는 과정입니다. 이를 "핸드셰이크(handshake)"라고도 합니다. 이 과정이 성공해야만 클라이언트는 서버가 정상적으로 실행되었고, 요청을 받을 준비가 되었음을 확신할 수 있습니다. await는 서버로부터 "준비 완료" 응답이 올 때까지 기다리라는 의미
            mcp_client = type("MCPClientHolder", (), {"session": session})()
                #  파이썬의 동적 클래스 생성 기능
            
            tools = await load_mcp_tools(session)
            
            agent = create_react_agent(llm, tools)
            
            print("MCP Client Started! Type 'quit' to exit.")
            while True:
                query = input("\nQuery: ").strip()
                if query.lower() == "quit":
                    break
                
                response = await agent.ainvoke({"messages": query})
                
                try:
                    formatted = json.dumps(response, indent=2, cls=CustomEncoder)
                except Exception as e:
                    formatted = str(response)
                print("\nResponse: ")
                print(formatted)
    return
            

```python 
mcp_client = type("MCPClientHolder", (), {"session": session})() 
```

1. 첫 번째 인자 : 클래스 이름
2. 두 번째 인자 : 상속받을 부모 클래스
3. 세 번째 인자 : 클래스가 가질 속성 딕셔너리
4. () : 바로 앞 tpye으로 만들어진 클래스의 인스턴스 생성

=

```python
# 1. 새로운 클래스를 정의합니다.
class MCPClientHolder:
    def __init__(self, session_object):
        self.session = session_object

# 2. 그 클래스의 인스턴스(객체)를 생성합니다.
mcp_client = MCPClientHolder(session_object=session)
```
